# 目的

#每日收盤後開始爬股價

#可以視情況條種是否抓營收，大概都隔月的10 後營收才會公布

## 【資料源】
來自 _BaseInfo  _CrawBase 會先去取基本相關資料 (股票號碼...等)

## 【輸出】
股票相關推薦圖表/數據

In [1]:
import pandas as pd

#  Update Base Info

In [2]:
 
#%run _BaseInfoETF.ipynb
%run _CrawBase.ipynb
#%run _LineMsg.ipynb
%run _genHtml.ipynb

C:\Users\user\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


%run RunRealTimeStock.ipynb

In [3]:
def MainStock(stock_number, start_month,end_time=datetime.now().strftime("%Y-%m-%d"),updateRevenue=False):  

    strmsg ='【stock_number】'+stock_number+'\n' +'【start_month】'+start_month+'\n'+'【end_time】'+end_time+'\n' 
    try:
        #處理 Stock
        RowData_df_craw_stock,His_Stock ,isSuccess= craw_stock(stock_number,start_month,end_time)
        #data_final,df_final=DataCleaning(RowData_df_craw_stock)
        print("【craw_stock】 success")
        
        #處理 Revenue
        if(updateRevenue):
            RowData_df_craw_revenue,His_Stock=craw_revenue(stock_number)
            df_Revenue=RevenueDataCleaning(RowData_df_craw_revenue)
            print("【craw_revenue】 success")
            
        #處理相關指標轉成html 
        stock_data=data_process(RowData_df_craw_stock)
    
        level,interval_type, lower_bound, upper_bound=get_interval(stock_data.iloc[-1]['收盤價'],np.array(stock_data['收盤價']))
        stock_data['stock_number']=stock_number
        stock_data['level']=level
        stock_data['interval_type']=interval_type
        stock_data['lower_bound']=lower_bound
        stock_data['upper_bound']=upper_bound
        save_df_to_excel(stock_data)
        gen_html(stock_number,stock_data)

        #sendLineMessage('成功'+'\n'+strmsg )
    except:
        #sendLineMessage('失败'+'\n'+strmsg)
        print('MainStock Fail'+'\n'+strmsg)
        
    #return data_final,df_final,df_Revenue
    

# 清資料使用
type_='Stock_His'
strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
df=get_DB()
df[~((df['status'] != 'OK')& (df['status'] != '很抱歉，沒有符合條件的資料!'))].to_csv('BaseInfo/His_Stock.csv',index=False) 
DDD=df[((df['status'] != 'OK')& (df['status'] != '很抱歉，沒有符合條件的資料!'))]['stock_number'].unique()

start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False
file_path = 'D:\Project\Jupyter\Stock\Main\DailyResultLast.xlsx'   

stock_number=str(6763)
if (want_update_Stock):        
    type_='Stock_His'
    strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
    delete_DB_stock_by_Month_Stock_number(strMonth,stock_number,type_)
try:
    if(want_update_Revenue):
        MainStock(stock_number,start_month,want_update_Revenue)
    else:
        MainStock(stock_number,start_month)
except Exception as error:
    print("An exception occurred:", error)
    print('Action Fail'+'\n')

start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False


for code in _baseInfo.dropna()['公司代號']:#.iloc[::-1]:
    stock_number=str(code)
    if (want_update_Stock):        
        type_='Stock_His'
        strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
        delete_DB_stock_by_Month_Stock_number(strMonth,stock_number,type_)
    try:
        if(want_update_Revenue):
            MainStock(stock_number,start_month,want_update_Revenue)
        else:
            MainStock(stock_number,start_month)
    except Exception as error:
        print("An exception occurred:", error)
        print('Action Fail'+'\n')

In [4]:
import threading
from datetime import datetime, timedelta

start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False

# 定義一個處理股票資料的函數，接收股票代號列表作為參數
def process_stock_codes(stock_codes):
    # 計算月份字串（僅需計算一次）
    strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')

    for code in stock_codes:
        stock_number = str(code)
        
        # 如果要更新歷史資料
        if want_update_Stock:        
            type_ = 'Stock_His'
            delete_DB_stock_by_Month_Stock_number(strMonth, stock_number, type_)
        
        try:
            # 根據 want_update_Revenue 的值來決定傳入的參數
            if want_update_Revenue:
                MainStock(stock_number, start_month, want_update_Revenue)
            else:
                MainStock(stock_number, start_month)
        except Exception as error:
            print(f"An exception occurred with stock {stock_number}: {error}")
            print('Action Fail\n')

# 分割資料，取前 500 個和其餘的股票代號

first_500_codes = _baseInfo[_baseInfo['Type']=='上市'].dropna()['公司代號']
remaining_codes =_baseInfo[_baseInfo['Type']!='上市'].dropna()['公司代號']

# 創建兩個執行緒，分別處理前 500 個和其餘股票代號
thread1 = threading.Thread(target=process_stock_codes, args=(first_500_codes,))
thread2 = threading.Thread(target=process_stock_codes, args=(remaining_codes,))

# 啟動執行緒
thread1.start()
thread2.start()

# 等待兩個執行緒完成
thread1.join()
thread2.join()

print("Stock processing completed.")

Stock_His 1240 2024-11-01 ===>外部抓取
Stock_His 1101 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1101
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=1240&date=2024/11/01&id=&response=json
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
茂生農經-農業科技業-上櫃
【craw_stock】 success
Stock_His 1259 2024-11-01 ===>外部抓取
log/DailyResult/DailyResult_2024-11-11.xlsx
台泥-水泥工業-上市
Stock_His 1102 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1102
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=1259&date=2024/11/01&id=&response=json
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
亞泥-水泥工業-上市
Stock_His 1103 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1103
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
安心-觀光事業-上櫃
Stock_His 1264 2024-11-01 ===>外部抓取
https:

【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
光洋科-其他電子業-上櫃
Stock_His 1788 2024-11-01 ===>外部抓取
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=1788&date=2024/11/01&id=&response=json
佳格-食品工業-上市
Stock_His 1229 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1229
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
杏昌-生技醫療業-上櫃
Stock_His 1796 2024-11-01 ===>外部抓取
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=1796&date=2024/11/01&id=&response=json
聯華-食品工業-上市
Stock_His 1231 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1231
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
金穎生技-食品工業-上櫃
Stock_His 1799 2024-11-01 ===>外部抓取
https://www.tpex.org.tw/www/zh-tw/afterTrading/trad

中石化-塑膠工業-上市
Stock_His 1315 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1315
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
正德-航運業-上櫃
Stock_His 2643 2024-11-01 ===>外部抓取
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=2643&date=2024/11/01&id=&response=json
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
達新-塑膠工業-上市
Stock_His 1316 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1316
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
捷迅-航運業-上櫃
Stock_His 2718 2024-11-01 ===>外部抓取
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=2718&date=2024/11/01&id=&response=json
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
上曜-建材營造-上市
Stock_His 1319 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1319
【craw_stock】 success
log/DailyResu

【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
誠品生活-文化創意業-上櫃
Stock_His 2937 2024-11-01 ===>外部抓取
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=2937&date=2024/11/01&id=&response=json
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
廣豐-其他-上市
Stock_His 1417 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1417
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
集雅社-居家生活-上櫃
Stock_His 2941 2024-11-01 ===>外部抓取
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=2941&date=2024/11/01&id=&response=json
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
嘉裕-紡織纖維-上市
Stock_His 1418 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1418
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
米斯特-居家生活-上櫃
Stock_His 2947 2024-11-01 ===>外部抓取
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradin

【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
宏和-紡織纖維-上市
Stock_His 1447 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1447
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
穩懋-半導體業-上櫃
Stock_His 3114 2024-11-01 ===>外部抓取
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3114&date=2024/11/01&id=&response=json
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
力鵬-紡織纖維-上市
Stock_His 1449 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=1449
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
好德-電子零組件業-上櫃
Stock_His 3115 2024-11-01 ===>外部抓取
https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock?code=3115&date=2024/11/01&id=&response=json
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
佳和-紡織纖維-上市
Stock_His 1451 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?respons

...

【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
華凌-光電業-上市
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-01-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-02-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-03-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-04-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-05-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-06-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-07-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-08-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-09-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6919 2023-10-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->St

【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
沛爾生醫-創-生技醫療業-上市
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-01-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-02-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-03-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-04-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-05-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-06-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-07-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-08-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-09-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6951 2023-10-01
An exception occurred: 'data'
【_CrawStock】 craw_stock erro

【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
成信實業-創-綠能環保-上市
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-01-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-02-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-03-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-04-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-05-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-06-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-07-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-08-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-09-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error -->Stock_His 6988 2023-10-01
An exception occurred: 'data'
【_CrawStock】 craw_stock error

勤誠-電腦及週邊設備業-上市
Stock_His 8213 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=8213
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
志超-電子零組件業-上市
Stock_His 8215 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=8215
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
明基材-光電業-上市
Stock_His 8222 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=8222
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
寶一-電機機械-上市
Stock_His 8249 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=8249
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
菱光-電子零組件業-上市
Stock_His 8261 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=8261
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-

【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
台汽電-油電燃氣業-上市
Stock_His 8940 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=8940
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
新天地-觀光事業-上市
Stock_His 8996 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=8996
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
高力-電機機械-上市
Stock_His 9802 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=9802
【craw_stock】 success
log/DailyResult/DailyResult2_2024-11-11.xlsx
鈺齊-KY-運動休閒-上市
Stock_His 9902 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=9902
【craw_stock】 success
log/DailyResult/DailyResult_2024-11-11.xlsx
台火-其他-上市
Stock_His 9904 2024-11-01 ===>外部抓取
http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20241101&stockNo=